best practices when programming a deep learning model. These practices mostly refer to how we can write organized, modularized, and extensible python code.

Most of them aren’t exclusive for machine learning applications but can be utilized on all sorts of python projects. But here we will see how we can apply them in deep learning using a hands-on programming approach.

Machine learning code is ordinary software and should always be treated as one. Therefore, as ordinary code it has a project structure, a documentation and design principles such as object-oriented programming.